All questions that are not listed here are in the PDF file

2

In [1]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('UBS.csv')

# Convert the 'Date' column to pandas datetime object and set it as the index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Compute daily returns
df['Returns'] = df['Adj Close'].pct_change()

# Subset the data for 2008
df_2008 = df.loc['2008-01-01':'2008-12-31']

# Compute the 1-day VaR for each day in 2008 using the historical method
VaR_1d = df_2008['Returns'].quantile(0.01)

# Multiply VaR by initial investment to get VaR in dollar terms
initial_investment = 1000000
VaR_1d_dollars = VaR_1d * initial_investment

print(f"1-day VaR at 99% confidence level in dollars: {VaR_1d_dollars:.2f}")


1-day VaR at 99% confidence level in dollars: -155615.61


3

In [2]:
import pandas as pd
import yfinance as yf
import numpy as np

# Download stock price data from Yahoo Finance
data = yf.download(["GS","UBS","JPM","C","BCS","MS","DB","BAC","BNPQY","CS"], start="2005-12-30", end="2009-01-01")

# Calculate daily returns
returns = data['Adj Close'].pct_change().dropna()

# Calculate 99% 1-day VaR for each bank in 2008 using historical method
VaR = pd.DataFrame(index=returns.loc['2008'].index, columns=returns.columns)
for date in returns.loc['2008'].index:
    for bank in returns.columns:
        VaR.loc[date, bank] = np.percentile(returns.loc[:date, bank], 1, interpolation='lower')


[*********************100%***********************]  10 of 10 completed


<ipython-input-2-8e3bf64b7f2d>:15: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they. (Deprecated NumPy 1.22)
  VaR.loc[date, bank] = np.percentile(returns.loc[:date, bank], 1, interpolation='lower')


In [3]:
VaR

,BAC,BCS,BNPQY,C,CS,DB,GS,JPM,MS,UBS
Date,,,,,,,,,,
2008-01-02,-0.034934,-0.049036,-0.042553,-0.047606,-0.044067,-0.042128,-0.044591,-0.038388,-0.055909,-0.046156
2008-01-03,-0.034934,-0.049036,-0.042553,-0.047606,-0.044067,-0.042128,-0.044591,-0.038388,-0.055909,-0.046156
2008-01-04,-0.034934,-0.049036,-0.042553,-0.047606,-0.044067,-0.042128,-0.044591,-0.038388,-0.055909,-0.046156
2008-01-07,-0.034934,-0.049036,-0.042553,-0.047606,-0.044067,-0.042128,-0.044591,-0.038388,-0.055909,-0.046156
2008-01-08,-0.036082,-0.049036,-0.042553,-0.047606,-0.044067,-0.042128,-0.044591,-0.039671,-0.055909,-0.046156
...,...,...,...,...,...,...,...,...,...,...
2008-12-24,-0.111765,-0.137445,-0.097345,-0.129811,-0.116636,-0.114474,-0.103097,-0.106364,-0.151989,-0.113358
2008-12-26,-0.111765,-0.137445,-0.097345,-0.129811,-0.116636,-0.114474,-0.103097,-0.106364,-0.151989,-0.113358
2008-12-29,-0.111765,-0.137445,-0.097345,-0.129811,-0.116636,-0.114474,-0.103097,-0.106364,-0.151989,-0.113358


In [5]:
from scipy.stats import norm

# Define portfolio weights
weights = {'GS': 0.12, 'UBS': 0.12, 'JPM': 0.12, 'C': 0.12, 'BCS': 0.12, 'MS': 0.12, 'DB': 0.06, 'BAC': 0.06, 'BNPQY': 0.06, 'CS': 0.06}

# Calculate 1-day VaR for each bank
data_returns = data['Adj Close'].pct_change().dropna()
var_1d = data_returns.apply(lambda x: np.percentile(x, 1))

# Calculate portfolio return for each bank on each day
portfolio_returns = data_returns * pd.Series(weights)

# Calculate portfolio loss at the VaR level for each bank
portfolio_losses = abs(portfolio_returns) * var_1d

# Calculate DVaR and CVaR for each bank
dvar = portfolio_losses.mean()
cvar = portfolio_losses[portfolio_losses >= dvar].mean()

print('DVaR for each bank:\n', dvar)
print('\nCVaR for each bank:\n', cvar)


DVaR for each bank:
 BAC     -0.000127
BCS     -0.000345
BNPQY   -0.000110
C       -0.000338
CS      -0.000134
DB      -0.000121
GS      -0.000227
JPM     -0.000238
MS      -0.000451
UBS     -0.000298
dtype: float64

CVaR for each bank:
 BAC     -0.000041
BCS     -0.000136
BNPQY   -0.000047
C       -0.000115
CS      -0.000055
DB      -0.000051
GS      -0.000098
JPM     -0.000091
MS      -0.000166
UBS     -0.000120
dtype: float64


Short Question 4

In [6]:
# Download daily data for the S&P500
data2 = yf.download("^GSPC", start="2016-04-25", end="2021-04-23")

# Calculate daily returns
data2['Return'] = data2['Close'].pct_change()

# Set parameters
invest_amount = 1e6 # $1m investment
n_simulations = 10000 # number of simulations
n_days = 2 # number of days to hold

# Simulate 2-day returns
simulated_returns = []
for i in range(n_simulations):
    # Draw two random indices
    index1 = np.random.randint(len(data2) - n_days)
    index2 = index1 + n_days
    
    # Calculate the 2-day return
    r = np.prod(1 + data2['Return'].iloc[index1:index2]) - 1
    simulated_returns.append(r)

# Calculate the VaR
var_99 = np.percentile(simulated_returns, 1)

# Calculate the 2-day potential loss
potential_loss = invest_amount * var_99

print("The estimated 2-day 99% VaR is:", round(var_99, 4))
print("The potential loss over 2 days is:", round(potential_loss, 2))

[*********************100%***********************]  1 of 1 completed
The estimated 2-day 99% VaR is: -0.0478
The potential loss over 2 days is: -47774.91
